#### Prerequisite

In [ ]:
!python -m pip install --upgrade pip

#### Create a docker image

This step can run in SageMaker classic notebook environment or in your own local environment with docker installed.

In [ ]:
# Image name below preferably starts with the prefix `sagemaker`
!docker build -t sagemaker_feature_encoder -f Dockerfile .

#### Push docker image from local to ECR

In [ ]:
%%sh

# Specify a name to your custom container
container_name=sagemaker_feature_encoder  # should match name from previous cell
echo "Container Name: " ${container_name}

# Retreive AWS account ID
account=$(aws sts get-caller-identity --query Account --output text)

# Get the AWS region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: " ${account}
echo "Region: "${region}

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "ECR Repository: " ${repository}

image="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:latest"
echo "ECR Image URI: " ${image}

# If the ECR repository does not exist, create it.
aws ecr describe-repositories --repository-names ${container_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${container_name} > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Tag the local image with ECR image name
docker tag ${container_name} ${image}

# Finally, push the local docker image to ECR with the full ECR image name
docker push ${image}

##### Uncomment and run the command below if your local images need to be cleaned.

In [ ]:
#!docker rmi $(docker images -q) -f

## Deploy feature encoder as a SageMaker real time endpoint

#### Imports 

In [ ]:
from time import gmtime, strftime
import sagemaker
import datetime
import boto3
import time

#### Essentials

In [ ]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
account = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name
sagemaker_client = boto3.client('sagemaker', region_name=region)
image_name = 'sagemaker_feature_encoder'
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest'
current_timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

#### Pack all encoders into a tar file and push to S3

In [ ]:
!tar -czf encoder.tar.gz -C ./data .

In [ ]:
default_bucket = session.default_bucket()
default_bucket

In [ ]:
!aws s3 cp encoder.tar.gz s3://{default_bucket}/

#### Create a SageMaker model object

In [ ]:
model_name = f'feature-encoder-{current_timestamp}'
model_artifacts_location = f's3://{default_bucket}/encoder.tar.gz'

In [ ]:
response = sagemaker_client.create_model(ModelName=model_name, 
                                         Containers=[{
                                             'Image': image_uri, 
                                             'Mode': 'SingleModel', 
                                             'ModelDataUrl': model_artifacts_location}], 
                                         ExecutionRoleArn=role)
response

#### Create a SageMaker endpoint configuration

In [ ]:
endpoint_config_name = f'feature-encoder-{current_timestamp}'

In [ ]:
response = sagemaker_client.create_endpoint_config(EndpointConfigName=endpoint_config_name, 
                                                   ProductionVariants=[{
                                                       'VariantName': 'v1', 
                                                       'ModelName': model_name, 
                                                       'InstanceType': 'ml.m5.xlarge', 
                                                       'InitialInstanceCount': 2, 
                                                       'InitialVariantWeight': 1
                                                   }])
response

#### Create a SageMaker endpoint

In [ ]:
endpoint_name = f'feature-encoder-{current_timestamp}'

In [ ]:
response = sagemaker_client.create_endpoint(EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)
response

#### Describe endpoint to track creation status

In [ ]:
describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response['EndpointStatus'] == 'Creating':
    response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    print(response['EndpointStatus'])
    time.sleep(15)

describe_endpoint_response

#### Invoke endpoint to test deployed feature encoder

In [ ]:
sagemaker_runtime = boto3.client('sagemaker-runtime', 
                                 region_name=region)

In [ ]:
raw_payload = b"10-19-2021,145,24,22,14,usa,65+"

In [ ]:
%%time

response = sagemaker_runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                             Body=raw_payload, 
                                             ContentType='text/csv')
response

##### Extract feature vector from the response

In [ ]:
feature_vector = response['Body'].read().decode('utf-8').strip()
feature_vector